# 🏀 QEPC Player Props Backtest

This notebook backtests player prop predictions against historical results.

## Features
- **Time-travel backtesting** - No lookahead bias
- **Multiple metrics** - Accuracy, Brier score, calibration
- **Confidence levels** - HIGH/MEDIUM/LOW predictions
- **Edge analysis** - Find betting opportunities

## Requirements
- `PlayerStatistics.csv` in `data/raw/` folder
- `player_props_engine.py` and `props_backtest_engine.py` in same folder or `props/` subfolder

In [1]:
"""
QEPC Player Props Backtest Notebook (Clean Version)

Run this inside your qepc_project environment.
"""

from pathlib import Path
import pandas as pd
import numpy as np
from datetime import date, timedelta

# ---------------------------------------------------------------------
# Imports from props package
# ---------------------------------------------------------------------
try:
    from props.player_props_engine import PlayerPropsEngine, PropPrediction
    from props.props_backtest_engine import PropsBacktestEngine, BacktestSummary
    print("✅ Props engines loaded successfully from props/ package!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Make sure:")
    print("  • Your notebook kernel started from the qepc_project root, OR")
    print("  • qepc_project is on sys.path via your QEPC bootstrap.")
    raise

# ---------------------------------------------------------------------
# Configuration
# ---------------------------------------------------------------------

# Let the engines auto-detect data/raw/PlayerStatistics.csv
DATA_PATH = None

BACKTEST_START = "2024-11-01"
BACKTEST_END   = "2024-11-15"

PROPS_TO_TEST = ['PTS', 'REB', 'AST', '3PM', 'PRA']
MIN_MINUTES   = 15.0

print(f"\n📊 Configuration:")
print(f"   Data path override: {DATA_PATH!r} (None = auto-detect)")
print(f"   Date Range: {BACKTEST_START} → {BACKTEST_END}")
print(f"   Props: {PROPS_TO_TEST}")

# ---------------------------------------------------------------------
# Quick Test – Single Player
# ---------------------------------------------------------------------

print("\n" + "=" * 60)
print("🧪 QUICK TEST: Single Player Prediction")
print("=" * 60)

# Initialize engine
engine = PlayerPropsEngine(DATA_PATH)  # or PlayerPropsEngine() with no args
engine.load_data()

# Choose a player that you know exists in PlayerStatistics.csv
test_player = "LeBron James"  # adjust as needed
test_opponent = "Golden State Warriors"  # match opponentteamName in your data

print(f"\n📈 Predictions for {test_player}:")
print("-" * 50)

for prop in ['PTS', 'REB', 'AST', 'PRA']:
    pred = engine.predict(
        player_name=test_player,
        prop_type=prop,
        opponent=test_opponent,
        is_home=True,
    )

    if pred:
        print(f"\n{prop}:")
        print(f"  Projection: {pred.projection:.2f}")
        print(f"  Range: {pred.floor:.1f} – {pred.ceiling:.1f}")
        print(f"  Confidence: {pred.confidence}")
        if prop == "PTS":
            print(f"  Over 25.5: {pred.over_prob(25.5):.1%}")
    else:
        print(f"\n{prop}: Player not found or insufficient data.")

# ---------------------------------------------------------------------
# Full Backtest
# ---------------------------------------------------------------------

print("\n" + "=" * 60)
print("🚀 RUNNING FULL BACKTEST")
print("=" * 60)

backtest = PropsBacktestEngine(DATA_PATH)

results = backtest.run_backtest(
    start_date=BACKTEST_START,
    end_date=BACKTEST_END,
    props=PROPS_TO_TEST,
    min_minutes=MIN_MINUTES,
    verbose=True,
)

print(f"\n✅ Backtest complete: {len(results)} predictions generated")

# ---------------------------------------------------------------------
# Summary
# ---------------------------------------------------------------------

print("\n" + "=" * 60)
print("📊 BACKTEST SUMMARY")
print("=" * 60)

if results:
    summary = backtest.get_summary()
    print(f"""
┌─────────────────────────────────────────────┐
│           OVERALL PERFORMANCE               │
├─────────────────────────────────────────────┤
│  Total Predictions:    {summary.total_predictions:>6}              │
│  Mean Absolute Error:  {summary.mean_absolute_error:>6.2f} pts     │
│  Median Abs Error:     {summary.median_absolute_error:>6.2f} pts   │
│  Mean % Error:         {summary.mean_pct_error*100:>6.1f}%         │
├─────────────────────────────────────────────┤
│           DIRECTIONAL ACCURACY              │
├─────────────────────────────────────────────┤
│  Overall Accuracy:     {summary.overall_accuracy*100:>6.1f}%       │
│  Over Accuracy:        {summary.over_accuracy*100:>6.1f}%          │
│  Under Accuracy:       {summary.under_accuracy*100:>6.1f}%         │
├─────────────────────────────────────────────┤
│           CALIBRATION                       │
├─────────────────────────────────────────────┤
│  Brier Score:          {summary.brier_score:>6.4f}                 │
│  (Lower is better, 0.25 = random)           │
├─────────────────────────────────────────────┤
│           BY CONFIDENCE                     │
├─────────────────────────────────────────────┤
│  HIGH Confidence:      {summary.high_conf_accuracy*100:>6.1f}%     │
│  MEDIUM Confidence:    {summary.medium_conf_accuracy*100:>6.1f}%   │
│  LOW Confidence:       {summary.low_conf_accuracy*100:>6.1f}%      │
├─────────────────────────────────────────────┤
│           SIMULATED BETTING                 │
├─────────────────────────────────────────────┤
│  Bets Placed:          {summary.simulated_bets:>6}                 │
│  Bets Won:             {summary.simulated_wins:>6}                 │
│  ROI:                  {summary.simulated_roi*100:>6.1f}%          │
└─────────────────────────────────────────────┘
    """)
else:
    print("❌ No results to summarize")


❌ Import error: No module named 'props'
Make sure:
  • Your notebook kernel started from the qepc_project root, OR
  • qepc_project is on sys.path via your QEPC bootstrap.


ModuleNotFoundError: No module named 'props'

In [ ]:
# Setup and Imports
import sys
from pathlib import Path

# Add props folder to path
for folder in [Path.cwd(), Path.cwd() / "props", Path.cwd().parent]:
    if (folder / "player_props_engine.py").exists():
        sys.path.insert(0, str(folder))
        break

import pandas as pd
import numpy as np
from datetime import date, timedelta

# Import engines
from player_props_engine import PlayerPropsEngine, PropPrediction
from props_backtest_engine import PropsBacktestEngine

print("✅ Props engines loaded!")

In [ ]:
import os
import sys
from pathlib import Path

print("🔍 Locating QEPC project root...\n")

# Try direct import first
try:
    from notebook_context import *
    print("✅ Imported notebook_context directly")
    
except ModuleNotFoundError:
    print("ℹ️  notebook_context not on path, searching...")
    
    # Search current directory and parents
    cwd = Path.cwd()
    candidates = [cwd, cwd.parent, cwd.parent.parent]
    
    found_root = None
    for root in candidates:
        if (root / "notebook_context.py").exists():
            found_root = root
            print(f"   Found at: {root}")
            break
    
    if found_root is None:
        raise FileNotFoundError(
            f"❌ Could not find notebook_context.py\n"
            f"   Searched: {cwd} and parent directories\n"
            f"   Ensure you're in the qepc_project folder"
        )
    
    # Add to path and re-import
    sys.path.insert(0, str(found_root))
    os.chdir(found_root)
    
    from notebook_context import *
    print("✅ Imported after path adjustment")

# Verify project_root is defined
try:
    project_root
except NameError:
    project_root = Path.cwd()
    print("⚠️  project_root not defined, using CWD")

print(f"\n📁 Project Root: {project_root}")
print(f"📂 Working Dir:  {os.getcwd()}")
print("\n" + "="*60)

In [ ]:
# Configuration
DATA_PATH = "data/raw/PlayerStatistics.csv"

# Backtest date range - EDIT THESE
BACKTEST_START = "2024-11-01"
BACKTEST_END = "2024-11-15"

# Props to test
PROPS_TO_TEST = ['PTS', 'REB', 'AST', '3PM', 'PRA']

print(f"📊 Date Range: {BACKTEST_START} → {BACKTEST_END}")
print(f"📊 Props: {PROPS_TO_TEST}")

In [ ]:
from pathlib import Path

# 1) Import from your props package
from props.player_props_engine import PlayerPropsEngine, PropsConfig

# 2) Point to your data folder (adjust if yours is different)
DATA_PATH = Path("data")

# 3) Initialize and load
engine = PlayerPropsEngine(DATA_PATH)
engine.load_data()

# 4) Simple test prediction
player_name = "Jayson Tatum"
prop_type = "PTS"
opponent = "Los Angeles Lakers"  # must match opponentteamName in PlayerStatistics

prediction = engine.predict(
    player_name=player_name,
    prop_type=prop_type,
    opponent=opponent,
    is_home=True,
)

print(prediction)
if prediction is not None:
    print("Projection:", prediction.projection)
    print("Std dev:   ", prediction.std_dev)


In [ ]:
# Run Full Backtest
print("🚀 Running backtest...")

backtest = PropsBacktestEngine(DATA_PATH)
results = backtest.run_backtest(
    start_date=BACKTEST_START,
    end_date=BACKTEST_END,
    props=PROPS_TO_TEST,
    verbose=True
)

print(f"\n✅ Complete: {len(results)} predictions")

In [ ]:
# Backtest Summary
summary = backtest.get_summary()

print("=" * 50)
print("📊 BACKTEST SUMMARY")
print("=" * 50)
print(f"Total Predictions: {summary.total_predictions}")
print(f"\n--- Accuracy ---")
print(f"Overall Accuracy:  {summary.overall_accuracy:.1%}")
print(f"Over Accuracy:     {summary.over_accuracy:.1%}")
print(f"Under Accuracy:    {summary.under_accuracy:.1%}")
print(f"\n--- Error ---")
print(f"Mean Abs Error:    {summary.mean_absolute_error:.2f} pts")
print(f"Median Abs Error:  {summary.median_absolute_error:.2f} pts")
print(f"\n--- Calibration ---")
print(f"Brier Score:       {summary.brier_score:.4f}")
print(f"(0.25 = random, lower is better)")
print(f"\n--- By Confidence ---")
print(f"HIGH:   {summary.high_conf_accuracy:.1%}")
print(f"MEDIUM: {summary.medium_conf_accuracy:.1%}")
print(f"LOW:    {summary.low_conf_accuracy:.1%}")
print(f"\n--- Simulated Betting ---")
print(f"Bets: {summary.simulated_bets}, Wins: {summary.simulated_wins}")
print(f"ROI: {summary.simulated_roi:.1%}")

In [ ]:
# Breakdown by Prop Type
prop_breakdown = backtest.breakdown_by_prop()
print("\n📈 BREAKDOWN BY PROP TYPE")
print("=" * 50)
display(prop_breakdown)

In [ ]:
# Detailed Results
df_results = backtest.results_to_dataframe()

print(f"\n📋 Sample Results ({len(df_results)} total)")
display(df_results.head(20))

In [ ]:
# Biggest Misses
print("\n⚠️ TOP 10 BIGGEST MISSES")
print("=" * 50)

biggest_misses = df_results.nlargest(10, 'abs_error')
display(biggest_misses[['player', 'date', 'prop', 'projection', 'actual', 'abs_error', 'confidence']])

In [ ]:
# Calibration Analysis
print("\n📐 PROBABILITY CALIBRATION")
print("=" * 50)
print("If well-calibrated: Predicted ≈ ActualOverRate")

calibration = backtest.calibration_analysis(bins=5)
display(calibration)

In [ ]:
# Export Results
from datetime import datetime

results_dir = Path("data/results/props_backtests")
results_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"props_backtest_{BACKTEST_START}_to_{BACKTEST_END}_{timestamp}.csv"
output_path = results_dir / filename

df_results.to_csv(output_path, index=False)
print(f"✅ Exported to: {output_path}")